This generates all the code in the library

In [4]:
HEX = "0123456789ABCDEF" * 16

In [ ]:
for i in range(16):
    print(f"/// This denotes the number {HEX[i]}")
    print(f"pub struct _{HEX[i]};")
    print(f"impl Hex for _{HEX[i]} {{}}")
    print()

In [ ]:
print("/// This trait is the equal trait on hex numbers. False evaluates to _0 and true evaluates to _1")
print("pub trait HexEqual<H: Hex> { type Output: Hex; }")
for i in range(16):
    for j in range(16):
        if i == j:
            print(f"impl HexEqual<_{HEX[i]}> for _{HEX[j]} {{ type Output = _1; }}")
        else:
            print(f"impl HexEqual<_{HEX[i]}> for _{HEX[j]} {{ type Output = _0; }}")

In [ ]:
print("/// This is an internal implementation of addition without carryover")
print("pub trait _HexAdd1<H: Hex> { type Output: Hex; type Carry: Hex; }")
for i in range(16):
    for j in range(16):
        print(f"impl _HexAdd1<_{HEX[i]}> for _{HEX[j]} {{ type Output = _{HEX[i + j]}; type Carry = _{HEX[(i+j)//16]}; }}")

In [ ]:
print("/// This is an internal implementation of addition with carryover")
print("pub trait _HexAdd2<H: Hex, C: Hex> { type Output: Hex; type Carry: Hex; }")
for i in range(16):
    for j in range(16):
        for k in range(16):
            res = i + j + k
            mods = res // 16
            print(f"impl _HexAdd2<_{HEX[i]}, _{HEX[j]}> for _{HEX[k]} {{ type Output = _{HEX[res]}; type Carry = _{HEX[mods]}; }}")